# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,djibo,14.1022,-1.6306,29.48,12,2,3.75,BF,1698717342
1,1,bredasdorp,-34.5322,20.0403,13.49,78,0,5.41,ZA,1698717342
2,2,prince george,53.9166,-122.7530,0.69,62,0,1.03,CA,1698717343
3,3,albany,42.6001,-73.9662,6.31,81,100,0.45,US,1698717343
4,4,colonia,50.9333,6.9500,11.65,86,100,7.72,DE,1698717280


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 36) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null value
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
26,26,rancho palos verdes,33.7445,-118.3870,24.64,30,0,1.79,US,1698717346
130,130,fortaleza,-3.7227,-38.5247,28.93,71,0,2.06,BR,1698717359
168,168,remire-montjoly,4.9167,-52.2667,27.02,94,0,1.03,GF,1698717127
195,195,adrar,20.5022,-10.0711,27.09,12,0,4.28,MR,1698717374
211,211,kuchaiburi,22.2667,86.1667,21.76,73,0,1.15,IN,1698717377
318,318,sanming,26.2486,117.6186,24.03,62,0,1.33,CN,1698717117
325,325,saint-pierre,-21.3393,55.4781,22.82,64,0,3.60,RE,1698717401
354,354,shenjiamen,29.9576,122.2980,21.98,60,0,2.83,CN,1698717406
373,373,tazacorte,28.6290,-17.9293,22.54,94,0,2.68,ES,1698717162
403,403,fort bragg,35.1390,-79.0060,22.91,66,0,3.60,US,1698717414


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country", "Lat","Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
26,rancho palos verdes,US,33.7445,-118.3870,30,
130,fortaleza,BR,-3.7227,-38.5247,71,
168,remire-montjoly,GF,4.9167,-52.2667,94,
195,adrar,MR,20.5022,-10.0711,12,
211,kuchaiburi,IN,22.2667,86.1667,73,
318,sanming,CN,26.2486,117.6186,62,
325,saint-pierre,RE,-21.3393,55.4781,64,
354,shenjiamen,CN,29.9576,122.2980,60,
373,tazacorte,ES,28.6290,-17.9293,94,
403,fort bragg,US,35.1390,-79.0060,66,


 Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary 
    #https://apidocs.geoapify.com/docs/geocoding/forward-geocoding/#api
    params["filter"] = f"circle:{longitude},{latitude},{radius}"

    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data   
hotel_df

Starting hotel search
rancho palos verdes - nearest hotel: No hotel found
fortaleza - nearest hotel: Hotel Catedral
remire-montjoly - nearest hotel: Complexe Belova
adrar - nearest hotel: No hotel found
kuchaiburi - nearest hotel: No hotel found
sanming - nearest hotel: 三明客家宾馆
saint-pierre - nearest hotel: Tropic Hotel
shenjiamen - nearest hotel: Zhonghan Hotel
tazacorte - nearest hotel: App Leyma
fort bragg - nearest hotel: Airborne Inn Lodging
jiwani - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
macapa - nearest hotel: Equatorial Hotel
al badari - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
hassi messaoud - nearest hotel: AMC
ubari - nearest hotel: فندق اوباري
dhromolaxia - nearest hotel: Helen of Troy gardens
al kharijah - nearest hotel: Kharga Hotel
rosarito - nearest hotel: WorldMark La Paloma


,City,Country,Lat,Lng,Humidity,Hotel Name
26,rancho palos verdes,US,33.7445,-118.3870,30,No hotel found
130,fortaleza,BR,-3.7227,-38.5247,71,Hotel Catedral
168,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
195,adrar,MR,20.5022,-10.0711,12,No hotel found
211,kuchaiburi,IN,22.2667,86.1667,73,No hotel found
318,sanming,CN,26.2486,117.6186,62,三明客家宾馆
325,saint-pierre,RE,-21.3393,55.4781,64,Tropic Hotel
354,shenjiamen,CN,29.9576,122.2980,60,Zhonghan Hotel
373,tazacorte,ES,28.6290,-17.9293,94,App Leyma
403,fort bragg,US,35.1390,-79.0060,66,Airborne Inn Lodging


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)